# 安装必备工具

In [ ]:
import subprocess

libraries_to_install = [

]

tools_to_install = [

    "iputils-ping",
    
]

installed_libraries = subprocess.run(["pip", "freeze"], capture_output=True).stdout.decode().split("\n")

# 尝试安装所有未安装的库
for library in libraries_to_install:
    if library == "black[jupyter]" and any("black" in installed_library for installed_library in installed_libraries):
        print(f"【 {library} 】已经安装，跳过安装")
        continue
    elif not any(library in installed_library for installed_library in installed_libraries):
        !pip install $library
    else:
        print(f"【 {library} 】已经安装，跳过安装")
        
# 尝试安装所有未安装的工具
update_needed = False
for tool in tools_to_install:
    exit_code = subprocess.run(["dpkg", "-s", tool], capture_output=True).returncode
    if exit_code != 0:
        update_needed = True
        break
    else:
        print(f"【 {tool} 】已经安装，跳过安装")
if update_needed:
    !apt-get update
    for tool in tools_to_install:
        exit_code = subprocess.run(["dpkg", "-s", tool], capture_output=True).returncode
        if exit_code != 0:
            !apt-get install -y $tool
    


# 从GitHub克隆OneClick-stable-diffusion项目

In [ ]:
# import os
# root_dir = os.path.abspath(os.sep)
# !cd $root_dir && git clone https://github.com/lcolok/OneClick-stable-diffusion

# 从GitHub克隆stable-diffusion-webui项目

In [ ]:
import subprocess
import os
from urllib.parse import urlparse

# 设置GitHub项目的URL
repo_url = "https://github.com/AUTOMATIC1111/stable-diffusion-webui.git"

def get_project_name_from_url(url: str) -> str:
    """
    获取GitHub项目的名称。
    :param url: GitHub项目的URL
    :return: GitHub项目的名称
    """
    # 解析URL
    parsed_url = urlparse(url)
    # 获取路径部分
    extPath = parsed_url.path
    (_, project_name) = os.path.split(extPath)
    
    if '.git' in project_name:
        project_name = project_name.split(".git")[0]
    
    return project_name

# 调用命令行工具git rev-parse，获取git项目的根目录
git_root = subprocess.run(
    ["git", "rev-parse", "--show-toplevel"],
    capture_output=True,
    text=True,
).stdout.strip()

# 获取根目录的父目录
parent_dir = os.path.dirname(git_root)

# 调用函数，获取目标路径的文件名
destination_path = get_project_name_from_url(repo_url)

# 将文件名与根目录的父目录拼接起来，得到完整的目标路径
destination_path = os.path.join(parent_dir, destination_path)

# 判断目标路径是否已存在
if os.path.exists(destination_path):
    # 如果目标路径已存在，判断它是否是目录
    if os.path.isdir(destination_path):
        # 如果是目录，则不执行git clone命令
        print("目标路径已存在并且是目录。跳过git clone。")
    else:
        # 如果不是目录，则抛出异常
        raise Exception("目标路径已存在，但不是目录。")
else:
    # 如果目标路径不存在，则执行git clone命令
    !cd $parent_dir && git clone $repo_url

# 2. 添加 conda 清华源

In [ ]:
# !source activate && conda env list &&\
# conda config --add channels https://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/free/ &&\
# conda config --add channels https://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/main/ &&\
# conda config --add channels https://mirrors.tuna.tsinghua.edu.cn/anaconda/cloud/conda-forge/ &&\
# conda config --add channels https://mirrors.tuna.tsinghua.edu.cn/anaconda/cloud/msys2/ &&\
# conda config --add channels https://mirrors.tuna.tsinghua.edu.cn/anaconda/cloud/pytorch/ &&\
# conda config --set show_channel_urls yes

# 3. 因为 webui 要求 py3.10，新建环境

In [ ]:
import subprocess

def install_conda_env(env_name):
    # 检查 conda 是否已经安装
    try:
        subprocess.run(["conda", "env", "list"], check=True)
    except subprocess.CalledProcessError:
        print("系统上没有安装 Conda。请先安装。")
        return

    # 检查环境是否已经存在
    env_exists = False
    output = subprocess.run(["conda", "env", "list"], capture_output=True).stdout.decode()
    for line in output.split("\n"):
        if env_name in line:
            env_exists = True
            break

    # 如果环境不存在，则创建新环境
    if env_exists:
        print(f"环境 {env_name} 已经存在，跳过创建。")
    else:
        subprocess.run(["conda", "create", "-n", env_name, "python=3.10.6", "-y"], check=True)
        print(f"已创建环境 {env_name}。")

install_conda_env("py3.10.6")


# 4. 用 os 模块的 system 函数执行 Bash 命令，要将新环境设置为默认启动

In [ ]:
import os

# 读取 /root/.bashrc 文件的内容
with open("/root/.bashrc", "r") as f:
    bashrc_content = f.read()

# 如果 /root/.bashrc 文件中包含了多余的 "conda activate py3.10.6" 这一行，就删除这些行
if "conda activate py3.10.6" in bashrc_content.splitlines():
    # 删除 "conda activate py3.10.6" 这一行
    bashrc_content = "\n".join([line for line in bashrc_content.splitlines() if line != "conda activate py3.10.6"])
    # 把修改后的内容写回 /root/.bashrc 文件
    with open("/root/.bashrc", "w") as f:
        f.write(bashrc_content)
# 否则，在 /root/.bashrc 文件末尾添加 "conda activate py3.10.6" 这一行
else:
    with open("/root/.bashrc", "a") as f:
        f.write("\nconda activate py3.10.6")

print("/root/.bashrc 已经修改好了！")

In [ ]:
!source activate && conda init bash && source /root/.bashrc

# 7. 安装依赖——tensorflow、pytorch、torchvision、cudatoolkit

In [ ]:
!source activate &&\
conda activate py3.10.6 &&\
pip install pip -U &&\
pip config set global.index-url "" &&\
conda install -y tensorflow=2.10.0 cudatoolkit=11.7 &&\
pip install torch torchvision --extra-index-url https://download.pytorch.org/whl/cu117
# conda install pytorch==1.12.1 torchvision==0.13.1 -c pytorch -c nvidia -y
# conda install pytorch torchvision pytorch-cuda=11.3 -c pytorch -c nvidia

# 9. 复制一个文件，不然以后启动webui会报错

In [ ]:
import os

# 查询本机 conda 安装的目录
conda_dir = os.popen("conda info --base").read().strip()
print(f"当前 conda 安装的目录：{conda_dir}")

# 设置 py3.10.6 环境的目录
env_dir = os.path.join(conda_dir, "envs", "py3.10.6")
print(f"py3.10.6 环境的目录：{env_dir}")

# 设置目标目录
target_dir = "/usr/lib/x86_64-linux-gnu/"

# 设置源文件路径
source_file = os.path.join(env_dir, 'lib', 'libstdc++.so.6.0.30')

# 设置目标文件路径
target_file = os.path.join(target_dir, 'libstdc++.so.6.0.30')

cmd=f"cp {source_file} {target_dir} && cd {target_dir} && rm libstdc++.so.6 && ln {target_file} libstdc++.so.6"
print(cmd)

# 执行命令行
os.system(cmd)

print("命令执行完成！")


# 查看一下位于env目录下的py3.10.6文件体积为多少

In [ ]:
import os

# 查询本机 conda 安装的目录
conda_dir = os.popen("conda info --base").read().strip()
print(f"当前 conda 安装的目录：{conda_dir}")

# 设置 conda 环境目录
env_dir = os.path.join(conda_dir, "envs")
print(f"conda 环境目录：{env_dir}")

# 设置 py3.10.6 环境的目录
py_316_dir = os.path.join(env_dir, "py3.10.6")
print(f"py3.10.6 环境的目录：{py_316_dir}")

!du -sh $py_316_dir

py_316_clone_dir='/output/py3.10.6'

if not os.path.exists(py_316_clone_dir):
    !cp -r $py_316_dir $py_316_clone_dir
else:
    print(f'{py_316_clone_dir} 已经复制了')
!du -sh $py_316_clone_dir

# 快速进行py3.10.6环境的挂载

In [ ]:
py_316_clone_dir='/output/py3.10.6'
if os.path.exists(py_316_dir):
    !rm -rf $py_316_dir
!ln -s $py_316_clone_dir $py_316_dir

In [ ]:
!conda env list

# 10.(可选) 测试一下tf和pytorch是否可用

In [ ]:
!source activate &&\
conda activate py3.10.6 &&\
python -c 'import tensorflow;tensorflow.test.is_gpu_available();' &&\
python -c 'import torch;torch.cuda.is_available();'


# 11. 删掉多余的缓存

In [ ]:
!rm -rf /usr/local/pkgs

# 12. 别忘记安装或者自行编译对应显卡核心代号的xformers！

## <i class="fa fa-bolt"></i> 安装Xformers

大幅减少计算时间和显存占用，代价是生成的图每个都有点变化，强烈推荐安装!  
但不同核心代号的显卡需要不同的编译包，不能混用  
当然此笔记会帮你自动检测这台机器是否适合安装，而且会自动帮你安装合适的编译包

In [ ]:
import os,sys
# 调用命令行工具git rev-parse，获取git项目的根目录，这样无论ipynb放在哪个位置，它都能找到根目录中的func
git_root = subprocess.run(
    ["git", "rev-parse", "--show-toplevel"],
    capture_output=True,
    text=True,
).stdout.strip()
# 把上级路径也加入到系统路径中，这样就能够找到func
sys.path.append(git_root) 

from func.xformers import installXformers
from func.env import setProxy
from IPython.display import clear_output

try:
    proxy,region
except NameError:
    cb=setProxy()
    proxy=cb['proxy']
    region=cb['region']
    clear_output(wait=True)

installXformers(proxy,region)

# 修改can_run_as_root参数，使脚本可以在root用户环境下运行

In [ ]:
import os
import re

pattern = re.compile(r'^([^\s=]+)\s*=\s*("*.*"*)$')

file_path = None
modified = False

root_dir = os.path.abspath(os.sep)
for root, dirs, files in os.walk(root_dir):
    for file in files:
        if file.endswith('.sh') and file == 'webui.sh':
            file_path = os.path.join(root, file)
            with open(file_path) as f:
                lines = f.readlines()
            with open(file_path, 'w') as f:
                for line in lines:
                    match = pattern.match(line)
                    if match and match.group(1) == 'can_run_as_root':                        
                        value = match.group(2)
                        if value == "0":
                            modified = True
                            line = f'{match.group(1)}={1}\n'
                        else:
                            modified = False
                    f.write(line)

# 检查修改是否成功
with open(file_path) as f:
    contents = f.read()
if modified and 'can_run_as_root=1' in contents:
    print(f'\033[32m文件 {file_path} 修改成功。\033[0m')
elif not modified and 'can_run_as_root=1' in contents:
    print(f'\033[32m文件 {file_path} 符合预期结果，但并不是本次执行的修改。\033[0m')
else:
    print(f'\033[31m文件 {file_path} 修改不成功。\033[0m')


# 为webui-user.sh添加各种args

In [ ]:
import os
import re

pattern = re.compile(r'(^\s*#*\s*export\s+COMMANDLINE_ARGS\s*=\s*".*")')
args = [
    '--share',
    '--opt-split-attention',
    '--port 6006',
    '--deepdanbooru',
    '--administrator',
    '--disable-safe-unpickle',
    '--xformers'
   ]
replacement = 'export COMMANDLINE_ARGS="' + ' '.join(args) + '"'

modified = False

root_dir = os.path.abspath(os.sep)
for root, dirs, files in os.walk(root_dir):
    for file in files:
        if file.endswith('.sh') and 'webui-user' in file:
            file_path = os.path.join(root, file)
            try:
                with open(file_path) as f:
                    lines = f.readlines()
                with open(file_path, 'w') as f:
                    for line in lines:
                        match = pattern.match(line)
                        if match:
                            target = match.group(1)
                            line = line.replace(target, replacement)
                            modified = True
                        f.write(line)
            except FileNotFoundError:
                print(f'\033[31m文件 {file_path} 不存在。\033[0m')

# 检查修改是否成功
for root, dirs, files in os.walk(root_dir):
    for file in files:
        if file.endswith('.sh') and 'webui-user' in file:
            file_path = os.path.join(root, file)
           
            with open(file_path) as f:
                contents = f.read()
            if replacement in contents:
                print(f'\033[32m文件 {file_path} 修改成功。\033[0m')
            else:
                print(f'\033[31m文件 {file_path} 修改不成功。\033[0m')


In [ ]:
!source activate && conda activate py3.10.6 && python -c 'import sys;print(sys.executable)'

# 挂载数据集中的ckpt到SD_WebUI模型所在的路径

In [ ]:
!ln -s /input1/FuXingHao768.ckpt /output/stable-diffusion-webui/models/Stable-diffusion/FuXingHao768.ckpt

In [ ]:
# !cp /input1/FuXingHao768.ckpt /output/stable-diffusion-webui/models/Stable-diffusion/FuXingHao768.ckpt